In [1]:
!git clone https://github.com/AI4Bharat/IndicTrans2
%cd IndicTrans2/huggingface_interface


Cloning into 'IndicTrans2'...
remote: Enumerating objects: 767, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 767 (delta 151), reused 105 (delta 104), pack-reused 585 (from 3)
Receiving objects: 100% (767/767), 4.16 MiB | 6.84 MiB/s, done.
Resolving deltas: 100% (493/493), done.
/content/IndicTrans2/huggingface_interface


In [2]:
!bash install.sh


Setting up the environment in the /content/IndicTrans2/huggingface_interface
Creating a virtual environment with python3
install.sh: line 10: conda: command not found
install.sh: line 11: conda: command not found
Installing all the dependencies
install.sh: line 14: conda: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  DEPRECATION: Building 'mosestokenizer' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'mosestokenizer'. 

In [10]:
# Install dependencies
!pip install transformers sentencepiece sacremoses IndicTransToolkit


In [17]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit.IndicTransToolkit import IndicProcessor

# 1. Select your device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 2. Set source and target language codes
src_lang = "eng_Latn"  # English in Latin script
tgt_lang = "hin_Deva"  # Hindi in Devanagari script (change as needed)

# 3. Define model name
model_name = "ai4bharat/indictrans2-en-indic-1B"

# 4. Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,         # faster & efficient if GPU supports
    attn_implementation="flash_attention_2"
).to(DEVICE)

# 5. Initialize the IndicProcessor for preprocessing & postprocessing
ip = IndicProcessor(inference=True)

# 6. Input sentences to translate
input_sentences = [
    "When I was young, I used to go to the park every day.",
    "We watched a new movie last week, which was very inspiring.",
    "My friend has invited me to his birthday party, and I will give him a gift.",
]

# 7. Preprocess the inputs
batch = ip.preprocess_batch(
    input_sentences,
    src_lang=src_lang,
    tgt_lang=tgt_lang,
)

# 8. Tokenize and prepare input tensors
inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)

# 9. Generate translations
with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=False,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )

# 10. Decode the outputs
generated_tokens = tokenizer.batch_decode(
    generated_tokens,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

# 11. Postprocess translations (handles entities, etc.)
translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

# 12. Display results
for src, tgt in zip(input_sentences, translations):
    print(f"EN → HI:")
    print(f"  Input:  {src}")
    print(f"  Output: {tgt}")
    print("—")


EN → HI:
  Input:  When I was young, I used to go to the park every day.
  Output: जब मैं छोटा था, मैं हर दिन पार्क जाता था।
—
EN → HI:
  Input:  We watched a new movie last week, which was very inspiring.
  Output: हमने पिछले हफ्ते एक नई फिल्म देखी, जो बहुत प्रेरणादायक थी।
—
EN → HI:
  Input:  My friend has invited me to his birthday party, and I will give him a gift.
  Output: मेरे दोस्त ने मुझे अपने जन्मदिन की पार्टी में आमंत्रित किया है, और मैं उसे एक उपहार दूंगा।
—
